In [4]:
import numpy as np
import metnum
from scoring import cross_validation as cv
from scoring import metrics

In [5]:
import pandas as pd

df_train = pd.read_csv("../data/train.csv")
df_train.sample(frac=1)
df_train = df_train[:42000]

In [6]:
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

In [7]:
%%time
alpha = 20
n_iter = 5000
eps = 1e-2

#pca = metnum.PCA(alpha)
#X = pca.transform(X, n_iter, eps)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [8]:
limit = int(0.8 * X.shape[0])

X_train, y_train = X[:limit], y[:limit]
X_val, y_val = X[limit:], y[limit:]

In [9]:
clf = metnum.KNNClassifier(10)
clf.fit(X_train, y_train)

In [ ]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_val)

acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))

## Cross validation ##

In [10]:
%%time
import os
import numpy as np

class MatrixFSDict:
    """Mantiene un FS dict de matrices de numpy"""

    def __init__(self, d):
        self.dir = d

    def load(self, name: str) -> np.ndarray:
        path = os.path.join(self.dir, name)
        print("loading:", path)
        return np.loadtxt(path, delimiter=',')
    
    def save(self, name: str, X: np.ndarray):
        path = os.path.join(self.dir, name)
        print("storing:", path)
        return np.savetxt(path, X, delimiter=",")
    
    def contains(self, name: str):
        return os.path.exists(os.path.join(self.dir, name))


CPU times: user 42 µs, sys: 0 ns, total: 42 µs
Wall time: 45.3 µs


In [11]:
DIR = "pca-cache"
MAX_ALPHA = 100

class CachedPCA:
    def __init__(self, alpha: int):
        assert(alpha < MAX_ALPHA)
        self.pca = metnum.PCA(alpha)
        self.cache = MatrixFSDict(DIR)
        self.alpha = alpha
        
    def fit(self, X, fold: int):
        name = "fold_{}".format(fold)
        if self.cache.contains(name):
            # HIT!
            print(f"Hit {fold}")
            tc = self.cache.load(name)
            # verificar!!!!!!!!!!!
            print(tc[:self.alpha+1].shape)
            self.pca.set_tc(tc[:self.alpha])
            return
        
        # MISS :(
        print(f"Miss {fold}")
        max_pca = metnum.PCA(MAX_ALPHA)
        max_pca.fit(X)
        tc = max_pca.get_tc()
        self.cache.save(name, tc)
    
        self.pca.set_tc(tc[:self.alpha+1])
        
    def transform(self, X):
        return self.pca.transform(X)

class KNNnPCAClassifier:
    def __init__(self, k, alpha):
        self.knn_classifier = metnum.KNNClassifier(k)
        self.pca = CachedPCA(alpha)

    def fit(self, X_train, y_train, fold=None):
        assert(fold != None)
        self.pca.fit(X_train, fold)
        self.knn_classifier.fit(self.pca.transform(X_train), y_train)
    
    def predict(self, X):
        return self.knn_classifier.predict(self.pca.transform(X))

In [ ]:
%%time
clf = KNNnPCAClassifier(10,20)
mean = cv.cross_validate_pca(clf, X, y, metrics.accuracy_score, 4, True)

Miss 0
storing: pca-cache/fold_0


In [32]:
clf = KNNnPCAClassifier(10,20)
clf.predict(X, **{"l_bound": 2, "r_bound": 1})

TypeError: predict() got an unexpected keyword argument 'l_bound'

In [10]:
mean

NameError: name 'mean' is not defined